In [1]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.curdir, '..')))
from src.utils import *
from src.ikflow_program import PandaIKProgram
from pydrake.all import (
    StartMeshcat,
)



ikflow/config.py | Using device: 'cuda:0'


In [2]:
meshcat = StartMeshcat()
diagram = BuildEnv(meshcat=meshcat, directives_file = os.path.join(RepoDir(), "models/panda/panda_collision.yaml"))

program = PandaIKProgram(diagram)
target_pose = np.array([-0.5, 0.5, 0.5, -0.6, 0.48, 0.64, 0]) #x, y, z, qw, qx, qy, qz
program.create_prog(target_pose)


# result = program.Solve()

# result.is_success()



INFO:drake:Meshcat listening for connections at http://localhost:7000


WorldModel::LoadRobot: /home/tangles/.cache/jrl/urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
Geometry: Loading 12 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae into Group
Geometry: Loading 4 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae into Group
Geometry: Loading 4 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae into Group
Geometry: Loading 3 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link5.dae into Group
Geometry: Loading 17 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link6.dae into Group
Geometry: Loa

In [3]:
vars = program.prog.GetInitialGuess(program.lumped_vars)
vars[7:] += 0
print(program.VarsToQ(vars))
print(program.EvalPositionError(vars))
print(program.EvalOrientationError(vars))


orientation = program.fk(program.VarsToQ(vars))[1]
print(orientation)
print(program.target_pose[3:])


position, orientation = program.fk(program.VarsToQ(vars))

print(np.concatenate([position, orientation], axis=0))
print(program.target_pose)
error = np.linalg.norm(np.concatenate([position, orientation], axis=0) - program.target_pose)
error

[-2.79245949  1.36949503 -1.7280513  -1.514112    1.98483217  2.1842804
  0.62595564  0.          0.        ]
[-0.09857466 -0.08825753 -0.02005145]
[0.78132365]
[ 0.4298189  -0.7384456  -0.48798752  0.1783872 ]
[-0.6   0.48  0.64  0.  ]
[-0.59857466  0.41174247  0.47994855  0.4298189  -0.7384456  -0.48798752
  0.1783872 ]
[-0.5   0.5   0.5  -0.6   0.48  0.64  0.  ]


1.9665255901791245

In [4]:
result = program.Solve()


print(result.is_success())

vars = result.get_x_val()

False


In [5]:
import torch
from ikflow.config import DEVICE
program.jacobian_gen(torch.tensor(vars, dtype=torch.float32, device=DEVICE, requires_grad=True))

tensor([[ 3.0023e+00, -9.2370e+00, -1.4372e+00, -2.7356e+00,  3.2619e-01,
         -5.1620e-01,  5.8541e-01,  1.9967e-01, -1.9594e+00,  1.9018e-01,
          5.2139e-03, -3.0791e-01,  3.0033e-02,  1.6520e-02],
        [ 4.2130e+00, -5.1559e+00,  6.7428e-01, -5.1102e-01,  9.5366e-02,
         -4.7740e-03,  4.9615e-02,  5.2233e-02, -5.7255e-01,  5.5376e-02,
          4.2074e-03, -9.1607e-02,  8.0797e-03,  7.2551e-03],
        [-5.6853e+00,  1.0613e+01,  1.8863e+00,  3.5100e+00, -5.2735e-01,
          5.7244e-01, -9.2501e-01, -2.3317e-01,  2.2885e+00, -2.1555e-01,
         -7.0363e-03,  3.5961e-01, -3.0570e-02, -1.5701e-02],
        [-3.9429e+00,  3.6360e+00,  1.1903e+00,  1.4873e-01,  1.0035e-01,
         -2.7364e-02,  2.2227e-01, -3.0655e-03,  6.8653e-02, -4.7483e-03,
         -4.1097e-03,  7.9483e-03, -2.8962e-03, -6.6315e-03],
        [ 9.7691e+00, -1.1543e+01,  6.6503e-01, -2.1350e+00, -6.5925e-01,
          1.2885e+00, -3.3290e+00,  2.3635e-01, -2.3854e+00,  2.3286e-01,
          1.